In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
RE = "Wind_Wallonie_Elia" # Solar_PBE Wind_Wallonie_Elia
address = "https://raw.githubusercontent.com/Jaeik-Jeong/DeepBid/main/"

data_train_csv1 = pd.read_csv(address+RE+'_16.csv', index_col=0)
data_train_csv2 = pd.read_csv(address+RE+'_17.csv', index_col=0)
data_train_csv  = pd.concat([data_train_csv1, data_train_csv2])
data_val_csv    = pd.read_csv(address+RE+'_18.csv', index_col=0)
data_test_csv   = pd.read_csv(address+RE+'_19.csv', index_col=0)

data_price = pd.read_csv(address+'Elia_Imbalance_Price_16_19.csv', index_col=0)
data_train_csv['Price(€)'] = data_price['Positive imbalance price'][:len(data_train_csv)]
data_val_csv['Price(€)']   = data_price['Positive imbalance price'][len(data_train_csv):len(data_train_csv)+len(data_val_csv)]
data_test_csv['Price(€)']  = data_price['Positive imbalance price'][len(data_train_csv)+len(data_val_csv):]

In [ ]:
# Data Preprocessing
 
Battery_Size = 0.15 #p.u.
unit         = 1 #unit: 15 minute
 
RE_Capacity1 = max(data_train_csv['Power(MW)'])
RE_Capacity2 = max(data_val_csv['Power(MW)'])
RE_Capacity3 = max(data_test_csv['Power(MW)'])
max_price = max(data_price['Marginal incremental price'])
 
size_train0 = int(len(data_train_csv)/unit)
size_val0   = int(len(data_val_csv)/unit)
size_test0  = int(len(data_test_csv)/unit)
 
data_train0 = []; data_train = []; price_train0 = []; price_train = [];
for i in range(size_train0):
    data_train0  += [round(pd.Series.mean(data_train_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity1, 3)]
    price_train0 += [round(pd.Series.mean(data_train_csv['Price(€)'][i*unit:(i+1)*unit])/max_price, 3)]
    if data_train0[i] > 0: data_train += [data_train0[i]]; price_train += [price_train0[i]]
 
data_val0 = []; data_val = []; price_val0 = []; price_val = []
for i in range(size_val0):
    data_val0  += [round(pd.Series.mean(data_val_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity2, 3)]
    price_val0 += [round(pd.Series.mean(data_val_csv['Price(€)'][i*unit:(i+1)*unit])/max_price, 3)]
    if data_val0[i] > 0: data_val += [data_val0[i]]; price_val += [price_val0[i]]
 
data_test0 = []; data_test = []; price_test0 = []; price_test = []
for i in range(size_test0):
    data_test0  += [round(pd.Series.mean(data_test_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity3, 3)]
    price_test0 += [round(pd.Series.mean(data_test_csv['Price(€)'][i*unit:(i+1)*unit])/max_price, 3)]
    if data_test0[i] > 0: data_test += [data_test0[i]]; price_test += [price_test0[i]]

In [ ]:
# PPO Agent (Partailly Observable State, Continuous Action Space)
# Assumption 1: Standard deviation is fixed
# Assumption 2: History is composed of observations only
 
n_layers         = 2
in_size          = 2
hidden_size      = 64
out_size         = 1
T_horizon        = 128
learning_rate    = 0.001
K_epoch          = 3
gamma            = 0.99
lmbda            = 0.95
eps_clip         = 0.01
C_value          = 1
var              = 0.1**2
 
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.fc_s  = nn.Linear(in_size, hidden_size)
        self.rnn   = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.fc_pi = nn.Linear(hidden_size, out_size)
        self.fc_v  = nn.Linear(hidden_size, 1)
 
    def pi(self, x, hidden):
        x = F.relu(self.fc_s(x))
        x = x.view(1, -1, hidden_size)
        x, hidden = self.rnn(x, hidden)
        pi = self.fc_pi(x)
        pi = pi.view(-1, out_size)
        return pi, hidden
    
    def v(self, x, hidden):
        x = F.relu(self.fc_s(x))
        x = x.view(1, -1, hidden_size)
        x, hidden = self.rnn(x, hidden)
        v = self.fc_v(x)
        v = v.view(-1, 1)
        return v
 
def train_net(model, batch, optimizer):
    o, H, a, r, o_prime, H_prime, done = [], [], [], [], [], [], []
    for transition in batch[0]:
        o.append(transition[0])
        a.append(transition[1])
        r.append([transition[2]])
        o_prime.append(transition[3])
        done.append([0]) if transition[4] else done.append([1])
    for transition in batch[1]:
        H.append(transition[0])
        H_prime.append(transition[1])
        
    o         = torch.tensor(o,dtype=torch.float)
    H         = (H[0][0].detach(), H[0][1].detach())
    a         = torch.tensor(a,dtype=torch.float)
    r         = torch.tensor(r,dtype=torch.float)
    o_prime   = torch.tensor(o_prime,dtype=torch.float)
    H_prime   = (H_prime[0][0].detach(), H_prime[0][1].detach())
    done      = torch.tensor(done)
 
    pdf_old = torch.distributions.MultivariateNormal(model.pi(o, H)[0], var*torch.eye(out_size))
    prob_old = torch.exp(pdf_old.log_prob(a)).view(len(a),1)
    prob_old = prob_old.detach()
 
    v_target = r + gamma * model.v(o_prime, H_prime) * done
    td = r + gamma * model.v(o_prime, H_prime) * done - model.v(o, H)
    td = td.detach().numpy()
    advantage = []
    A = 0.0
    for delta in td[::-1].flatten():
        A = delta + gamma*lmbda*A
        advantage.append([A])
    advantage.reverse()
    advantage = torch.tensor(advantage, dtype=torch.float)
    
    for i in range(K_epoch):
        pdf = torch.distributions.MultivariateNormal(model.pi(o, H)[0], var*torch.eye(out_size))
        prob = torch.exp(pdf.log_prob(a)).view(len(a),1)
        ratio = torch.exp(torch.log(prob) - torch.log(prob_old))  # a/b == exp(log(a)-log(b))
 
        loss_actor = torch.min(ratio * advantage, torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage)
        loss_critic = F.mse_loss(model.v(o, H), v_target.detach())
        loss = -(loss_actor - C_value*loss_critic)
        
        optimizer.zero_grad()
        loss.mean().backward(retain_graph=True)
        optimizer.step()

In [ ]:
# Environment
 
E_max   = Battery_Size
P_max   = E_max
tdelta  = unit/4
soc_min = 0.1
soc_max = 0.9
a0 = -1.031; a1 = 35; a2 = 3.685; a3 = 0.2156; a4 = 0.1178; a5 = 0.3201
b0 = 0.1463; b1 = 30.27; b2 = 0.1037; b3 = 0.0584; b4 = 0.1747; b5 = 0.1288
c0 = 0.1063; c1 = 62.49; c2 = 0.0437; d0 = 0.0712; d1 = 61.4; d2 = 0.0288
N = 130*215*E_max/0.1
beta = 10/max_price
 
class Env():
    def __init__(self, data, price):
        self.data = data
        self.price = price
        self.state = []
 
    def reset(self):
        gen = self.data[0]
        E = E_max/2
        state = [[gen, E]]
        self.state = state
        return state
 
    def step(self, action):
        gen = self.data[len(self.state)]
        bid = action[0]
        imb = self.price[len(self.state)]

        E = self.state[-1][-1]
        soc = E/E_max
        Voc = a0*np.exp(-a1*soc) + a2 + a3*soc - a4*soc**2 + a5*soc**3
        Rs  = b0*np.exp(-b1*soc) + b2 + b3*soc - b4*soc**2 + b5*soc**3
        Rts = c0*np.exp(-c1*soc) + c2
        Rtl = d0*np.exp(-d1*soc) + d2
        R   = Rs + Rts + Rtl

        I_cmax = 1000000*(E_max*soc_max - E)/N/(Voc*tdelta)
        I_dmax = 1000000*(E - E_max*soc_min)/N/(Voc*tdelta)
        p_cmax = N*(Voc*I_cmax + I_cmax**2*R)
        p_dmax = N*(Voc*I_dmax - I_dmax**2*R)

        P_cmax = p_cmax/1000000; P_dmax = p_dmax/1000000
        P_c = min(max(gen-bid, 0), P_max, P_cmax)
        P_d = min(max(bid-gen, 0), P_max, P_dmax)
        p_c = 1000000*P_c/N; p_d = 1000000*P_d/N

        I_c = -(Voc - np.sqrt(Voc**2 + 4*R*p_c))/(2*R)
        I_d = (Voc - np.sqrt(Voc**2 - 4*R*p_d))/(2*R)
        if not np.isclose(p_c, 0):
            eff_c = (Voc*I_c)/p_c; eff_d = 1
            E_prime = E + eff_c*P_c*tdelta
            disp = gen - P_c
        elif not np.isclose(p_d, 0):
            eff_d = p_d/(Voc*I_d); eff_c = 1
            E_prime = E - (1/eff_d)*P_d*tdelta
            disp = gen + P_d
        else:
            eff_c = 1; eff_d = 1
            E_prime = E
            disp = gen

        error = bid - disp
        error_function = abs(error) + beta*P_c + beta*P_d
        revenue = (imb*disp - imb*abs(bid-disp) - beta*(P_c+P_d))*tdelta
 
        next_state = state + [[gen, E_prime]]
        reward = -error_function
        done = False
        info = [gen, bid, disp, revenue]
 
        self.state = next_state
        return next_state, reward, done, info

In [ ]:
# PPO Training

total_episode = 500
max_iteration = int(len(data_train)/T_horizon)
print_interval = 1
 
model = LSTM()
env_train = Env(data_train, price_train)
env_val   = Env(data_val, price_val)
env_test  = Env(data_test, price_test)
bid_train, bid_val, bid_test = [], [], [] # Bidding Value
mae_train, mae_val, mae_test = [], [], [] # Mean Absolute Error
mbe_train, mbe_val, mbe_test = [], [], [] # Mean Bidding Error
rev_train, rev_val, rev_test = [], [], [] # Revenue

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for n_epi in range(total_episode):
    bid_train += [[]]; bid_val += [[]]; bid_test += [[]]
    mae_train += [[]]; mae_val += [[]]; mae_test += [[]]
    mbe_train += [[]]; mbe_val += [[]]; mbe_test += [[]]
    rev_train += [[]]; rev_val += [[]]; rev_test += [[]]
 
    state = env_train.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for i in range(max_iteration):
        batch = [[],[]]
        for t in range(T_horizon):
            pi_out, next_history = model.pi(torch.tensor(state[-1], dtype=torch.float), history)
            action = np.random.multivariate_normal(pi_out.detach().numpy()[0], var*np.identity(out_size), 1)[0].tolist()
            next_state, reward, done, info = env_train.step(action)
 
            batch[0].append((state[-1], action, reward, next_state[-1], done))
            batch[1].append((history, next_history))
            state = next_state[:]
            history = next_history
 
            gen = info[0]; bid = info[1]; disp = info[2]; revenue = info[3]
            bid_train[n_epi] += [bid]
            mae_train[n_epi] += [abs(gen - bid)]
            mbe_train[n_epi] += [abs(disp - bid)]
            rev_train[n_epi] += [revenue]
            if done:
                break
        
        if n_epi != 0:
            train_net(model, batch, optimizer)
        if done:
            break
    
    state = env_val.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for k in range(len(env_val.data)-1):
        pi_out, next_history = model.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_val.step(action)
 
        state = next_state[:]
        history = next_history
        
        gen = info[0]; bid = info[1]; disp = info[2]; revenue = info[3]
        bid_val[n_epi] += [bid]
        mae_val[n_epi] += [abs(gen - bid)]
        mbe_val[n_epi] += [abs(disp - bid)]
        rev_val[n_epi] += [revenue]
    
    state = env_test.reset()
    history = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    for l in range(len(env_test.data)-1):
        pi_out, next_history = model.pi(torch.tensor(state[-1], dtype=torch.float), history)
        action = pi_out[0].tolist()
        next_state, reward, done, info = env_test.step(action)
 
        state = next_state[:]
        history = next_history
        
        gen = info[0]; bid = info[1]; disp = info[2]; revenue = info[3]
        bid_test[n_epi] += [bid]
        mae_test[n_epi] += [abs(gen - bid)]
        mbe_test[n_epi] += [abs(disp - bid)]
        rev_test[n_epi] += [revenue]
    
    if (n_epi+1)%print_interval == 0:
        MAE_train = round(100*np.mean(mae_train[n_epi]),2)
        MAE_val   = round(100*np.mean(mae_val[n_epi]),2)
        MAE_test  = round(100*np.mean(mae_test[n_epi]),2)
        MBE_train = round(100*np.mean(mbe_train[n_epi]),2)
        MBE_val   = round(100*np.mean(mbe_val[n_epi]),2)
        MBE_test  = round(100*np.mean(mbe_test[n_epi]),2)
        REV_train = round(max_price*RE_Capacity1*np.mean(rev_train[n_epi]),3)
        REV_val   = round(max_price*RE_Capacity2*np.mean(rev_val[n_epi]),3)
        REV_test  = round(max_price*RE_Capacity3*np.mean(rev_test[n_epi]),3)
 
        print("episode: {}".format(n_epi+1))
        print("MAE_train: {}%".format(MAE_train).ljust(25), end="")
        print("MAE_val: {}%".format(MAE_val).ljust(25), end="")
        print("MAE_test: {}%".format(MAE_test).ljust(25))
        print("MBE_train: {}%".format(MBE_train).ljust(25), end="")
        print("MBE_val: {}%".format(MBE_val).ljust(25), end="")
        print("MBE_test: {}%".format(MBE_test).ljust(25))
        print("REV_train: ${}".format(REV_train).ljust(25), end="")
        print("REV_val: ${}".format(REV_val).ljust(25), end="")
        print("REV_test: ${}".format(REV_test).ljust(25))
        print("------------------------------------------------------------------------------------------")

episode: 1
MAE_train: 30.59%        MAE_val: 30.55%          MAE_test: 32.57%         
MBE_train: 30.0%         MBE_val: 30.55%          MBE_test: 32.57%         
REV_train: $-89.994      REV_val: $-116.355       REV_test: $-87.196       
------------------------------------------------------------------------------------------
episode: 2
MAE_train: 21.95%        MAE_val: 13.48%          MAE_test: 15.03%         
MBE_train: 19.57%        MBE_val: 12.76%          MBE_test: 14.21%         
REV_train: $13.974       REV_val: $108.306        REV_test: $82.54         
------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [ ]:
# Environment
 
select_num = np.argmin(np.mean(mbe_val[:-1],axis=1))
select_test = np.array(bid_test[select_num][:])
select_test_real = np.array(data_test[1:])
select_test_price = np.array(price_test[1:])
 
E = E_max/2
mbe = []
reward = []
info = []
for i in range(len(select_test)):
    bid = select_test[i]
    gen = select_test_real[i]
    imb = select_test_price[i]
    
    soc = E/E_max
    Voc = a0*np.exp(-a1*soc) + a2 + a3*soc - a4*soc**2 + a5*soc**3
    Rs  = b0*np.exp(-b1*soc) + b2 + b3*soc - b4*soc**2 + b5*soc**3
    Rts = c0*np.exp(-c1*soc) + c2
    Rtl = d0*np.exp(-d1*soc) + d2
    R   = Rs + Rts + Rtl
 
    I_cmax = 1000000*E_max*(soc_max - soc)/N/(Voc*tdelta)
    I_dmax = 1000000*E_max*(soc - soc_min)/N/(Voc*tdelta)
    p_cmax = N*(Voc*I_cmax + I_cmax**2*R)
    p_dmax = N*(Voc*I_dmax - I_dmax**2*R)
 
    P_cmax = p_cmax/1000000; P_dmax = p_dmax/1000000
    P_c = min(max(gen-bid, 0), P_max, P_cmax)
    P_d = min(max(bid-gen, 0), P_max, P_dmax)
    p_c = 1000000*P_c/N; p_d = 1000000*P_d/N
 
    I_c = -(Voc - np.sqrt(Voc**2 + 4*R*p_c))/(2*R)
    I_d = (Voc - np.sqrt(Voc**2 - 4*R*p_d))/(2*R)
    if not np.isclose(p_c, 0):
        eff_c = (Voc*I_c)/p_c
        E = E + eff_c*P_c*tdelta
        disp = gen - P_c
        info += [[gen, round(bid,4), 'C', round(P_c,4), round(disp,4), round(eff_c,4), round(E,4)]]
    elif not np.isclose(p_d, 0):
        eff_d = p_d/(Voc*I_d)
        E = E - (1/eff_d)*P_d*tdelta
        disp = gen + P_d
        info += [[gen, round(bid,4), 'D', round(P_d,4), round(disp,4), round(eff_d,4), round(E,4)]]
    else:
        disp = gen
        info += [[gen, round(bid,4), 'N', 'N', round(disp,4), 'N', round(E,4)]]
    
    mbe += [abs(bid - disp)]
    reward += [(imb*disp - imb*abs(bid-disp) - beta*(P_c+P_d))*tdelta]
 
MAE_test = round(100*np.mean(np.abs(select_test_real - select_test)),2)
MBE_test = round(100*np.mean(mbe),2)
print("MAE_test: {}%".format(MAE_test))
print("MBE_test: {}%".format(MBE_test))
print("REV_test: ${}".format(round(max_price*RE_Capacity3*np.mean(reward),3)))
 
# pd.DataFrame(select_test).to_csv(RE+"_Model3_DeepComp.csv")

MAE_test: 15.03%
MBE_test: 14.21%
REV_test: $82.54
